In [8]:
import gc
import warnings
warnings.filterwarnings('ignore')
import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
import itertools
from itertools import combinations

import random
import joblib
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

from pd.gmb_utils import lgb_amex_metric
from pd.params import *



In [2]:
from pd.data.preprop import get_feat_comb
get_feat_comb()

In [ ]:
def get_agg_data(data_dir="train_agg_mean_q5_q95_q5_q95.npz"):
    d = np.load(OUTDIR+data_dir)
    #train_data = np.concatenate((d["d2"].astype(np.int32), d["d1"].reshape(d["d1"].shape[0], -1)), axis=1)
    train_labels = d["labels"]
    df2 = pd.DataFrame(d["d2"].astype(np.int32))
    df = pd.DataFrame(d["d1"].reshape(d["d1"].shape[0], -1))
    df = pd.concat((df2, df), axis=1,)
    df.columns = [f"c{i}" for i in range(df.shape[1])]
    cat_indices = list(np.arange(d["d2"].shape[1]))

    return df, train_labels, cat_indices

In [82]:
params = {
        'objective': 'binary',
        'metric': "binary_logloss",
        'boosting': 'dart',
        'seed': 42,
        'num_leaves': 100,
        'learning_rate': 0.01,
        'feature_fraction': 0.20,
        'bagging_freq': 10,
        'bagging_fraction': 0.50,
        'n_jobs': -1,
        'lambda_l2': 4,
        'min_data_in_leaf': 40
        }

In [50]:
#train_data = np.load(OUTDIR+"train_logistic_raw_all_mean_q5_q95_q5_q95_data.npy")
#train_labels = np.load(OUTDIR+"train_logistic_raw_all_mean_q5_q95_q5_q95_labels.npy")
#lgb_train = lgb.Dataset(X_train.reshape(X_train.shape[0], -1), y_train, categorical_feature=cat_indices)
#lgb_valid = lgb.Dataset(X_test.reshape(X_test.shape[0], -1), y_test, categorical_feature=cat_indices)


In [91]:

X_train, X_test, y_train, y_test = train_test_split(df, train_labels, test_size=1/9, random_state=0, shuffle=True)
validation_data = (X_test, y_test)


In [94]:
lgb_train = lgb.Dataset(X_train, y_train,)
lgb_valid = lgb.Dataset(X_test, y_test, )

In [95]:
model = lgb.train(
            params = params,
            train_set = lgb_train,
            num_boost_round = 5500,
            valid_sets = [lgb_train, lgb_valid],
            early_stopping_rounds = 100,
            verbose_eval = 100,
            feval = lgb_amex_metric, 
            )
        

[LightGBM] [Info] Number of positive: 105712, number of negative: 302210
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.903616 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 324251
[LightGBM] [Info] Number of data points in the train set: 407922, number of used features: 2322
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.259148 -> initscore=-1.050404
[LightGBM] [Info] Start training from score -1.050404
[100]	training's binary_logloss: 0.471143	training's amex_metric gini 0.905 recall 0.607: 0.755657	valid_1's binary_logloss: 0.470878	valid_1's amex_metric gini 0.904 recall 0.606: 0.754691
[200]	training's binary_logloss: 0.451443	training's amex_metric gini 0.906 recall 0.613: 0.759837	valid_1's binary_logloss: 0.45151	valid_1's amex_metric gini 0.905 recall 0.609: 0.756774
[300]	training's binary_logloss: 0.398248	training's 

In [96]:
model.save_model(OUTDIR+"lgbm_agg")

In [7]:
params = {
        'objective': 'binary',
        'metric': "binary_logloss",
        'boosting': 'dart',
        'seed': 42,
        'num_leaves': 100,
        'learning_rate': 0.001,
        'feature_fraction': 0.20,
        'bagging_freq': 10,
        'bagging_fraction': 0.50,
        'n_jobs': -1,
        'lambda_l2': 8,
        'min_data_in_leaf': 40
        }

In [8]:
lgb_train = lgb.Dataset(X_train.reshape(X_train.shape[0], -1), y_train)
lgb_valid = lgb.Dataset(X_test.reshape(X_test.shape[0], -1), y_test,)
model2 = lgb.train(
            params = params,
            train_set = lgb_train,
            num_boost_round = 5000,
            valid_sets = [lgb_train, lgb_valid],
            early_stopping_rounds = 100,
            verbose_eval = 100,
            feval = lgb_amex_metric
            )
        

[LightGBM] [Info] Number of positive: 105541, number of negative: 302381
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 1.270654 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 329344
[LightGBM] [Info] Number of data points in the train set: 407922, number of used features: 2443
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258728 -> initscore=-1.052588
[LightGBM] [Info] Start training from score -1.052588
[100]	training's binary_logloss: 0.593543	training's amex_metric gini 0.900 recall 0.592: 0.746145	valid_1's binary_logloss: 0.594411	valid_1's amex_metric gini 0.897 recall 0.583: 0.740103
[200]	training's binary_logloss: 0.609187	training's amex_metric gini 0.901 recall 0.594: 0.74746	valid_1's binary_logloss: 0.609827	valid_1's amex_metric gini 0.898 recall 0.586: 0.742201
[300]	training's binary_logloss: 0.600042	training's 

KeyboardInterrupt: 

In [6]:
lgb_train = lgb.Dataset(X_train.reshape(X_train.shape[0], -1), y_train)
lgb_valid = lgb.Dataset(X_test.reshape(X_test.shape[0], -1), y_test,)
model = lgb.train(
            params = params,
            train_set = lgb_train,
            num_boost_round = 10500,
            valid_sets = [lgb_train, lgb_valid],
            early_stopping_rounds = 100,
            verbose_eval = 100,
            feval = lgb_amex_metric
            )
        

[LightGBM] [Info] Number of positive: 105541, number of negative: 302381
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 1.232630 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 329344
[LightGBM] [Info] Number of data points in the train set: 407922, number of used features: 2443
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258728 -> initscore=-1.052588
[LightGBM] [Info] Start training from score -1.052588
[100]	training's binary_logloss: 0.470607	training's amex_metric gini 0.903 recall 0.604: 0.753508	valid_1's binary_logloss: 0.472209	valid_1's amex_metric gini 0.900 recall 0.592: 0.745712
[200]	training's binary_logloss: 0.450727	training's amex_metric gini 0.905 recall 0.610: 0.757501	valid_1's binary_logloss: 0.452516	valid_1's amex_metric gini 0.901 recall 0.598: 0.74981
[300]	training's binary_logloss: 0.397729	training's 

In [9]:
lgb_train = lgb.Dataset(X_train.reshape(X_train.shape[0], -1), y_train)
lgb_valid = lgb.Dataset(X_test.reshape(X_test.shape[0], -1), y_test,)
model = lgb.train(
            params = params,
            train_set = lgb_train,
            num_boost_round = 1500,
            valid_sets = [lgb_train, lgb_valid],
            early_stopping_rounds = 100,
            verbose_eval = 100,
            feval = lgb_amex_metric
            )
        

[LightGBM] [Info] Number of positive: 79627, number of negative: 263514
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 1.150216 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 328368
[LightGBM] [Info] Number of data points in the train set: 343141, number of used features: 2425
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.232053 -> initscore=-1.196753
[LightGBM] [Info] Start training from score -1.196753
[100]	training's binary_logloss: 0.447653	training's amex_metric gini 0.918 recall 0.656: 0.787208	valid_1's binary_logloss: 0.449236	valid_1's amex_metric gini 0.911 recall 0.631: 0.770685
[200]	training's binary_logloss: 0.430692	training's amex_metric gini 0.919 recall 0.661: 0.790093	valid_1's binary_logloss: 0.433192	valid_1's amex_metric gini 0.911 recall 0.635: 0.77314
[300]	training's binary_logloss: 0.3763	training's ame

In [12]:
lgb_train = lgb.Dataset(X_train.reshape(X_train.shape[0], -1), y_train)
lgb_valid = lgb.Dataset(X_test.reshape(X_test.shape[0], -1), y_test,)
model = lgb.train(
            params = params,
            train_set = lgb_train,
            num_boost_round = 10500,
            valid_sets = [lgb_train, lgb_valid],
            early_stopping_rounds = 100,
            verbose_eval = 100,
            feval = lgb_amex_metric
            )
        

[LightGBM] [Info] Number of positive: 79627, number of negative: 263514
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 1.282570 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 328368
[LightGBM] [Info] Number of data points in the train set: 343141, number of used features: 2425
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.232053 -> initscore=-1.196753
[LightGBM] [Info] Start training from score -1.196753
[100]	training's binary_logloss: 0.448219	training's amex_metric gini 0.917 recall 0.652: 0.784676	valid_1's binary_logloss: 0.449789	valid_1's amex_metric gini 0.910 recall 0.628: 0.769098
[200]	training's binary_logloss: 0.431256	training's amex_metric gini 0.919 recall 0.658: 0.788337	valid_1's binary_logloss: 0.433742	valid_1's amex_metric gini 0.911 recall 0.635: 0.772682
[300]	training's binary_logloss: 0.376935	training's 

In [13]:

joblib.dump(model, OUTDIR+f'lgbm13.pkl')


['/Users/nimamanaf/Desktop/kaggle/pd/data/out/lgbm13.pkl']

In [11]:
def train_and_evaluate(train, test=None, n_folds=5, seed=42):
    cat_features = [f"{cf}_last" for cf in CATCOLS]
    for cat_col in cat_features:
        encoder = LabelEncoder()
        train[cat_col] = encoder.fit_transform(train[cat_col])
        if test is not None:
            test[cat_col] = encoder.transform(test[cat_col])
    # Round last float features to 2 decimal place
    num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
    num_cols = [col for col in num_cols if 'last' in col]
    for col in num_cols:
        train[col + '_round2'] = train[col].round(2)
        if test is not None:
            test[col + '_round2'] = test[col].round(2)
    # Get feature list
    features = [col for col in train.columns if col not in ['customer_ID', "S_2", "target"]]
    
    # Create a numpy array to store test predictions
    test_predictions = np.zeros(len(test))
    # Create a numpy array to store out of folds predictions
    oof_predictions = np.zeros(len(train))
    kfold = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train["target"])):
        print(' ')
        print('-'*50)
        print(f'Training fold {fold} with {len(features)} features...')
        x_train, x_val = train[features].iloc[trn_ind], train[features].iloc[val_ind]
        y_train, y_val = train["target"].iloc[trn_ind], train["target"].iloc[val_ind]
        lgb_train = lgb.Dataset(x_train, y_train, categorical_feature = cat_features)
        lgb_valid = lgb.Dataset(x_val, y_val, categorical_feature = cat_features)
        model = lgb.train(
            params = params,
            train_set = lgb_train,
            num_boost_round = 10500,
            valid_sets = [lgb_train, lgb_valid],
            early_stopping_rounds = 100,
            verbose_eval = 100,
            feval = lgb_amex_metric
            )
        # Save best model
        joblib.dump(model, OUTDIR+f'Models/lgbm_fold{fold}_seed{seed}.pkl')
        val_pred = model.predict(x_val) # Predict validation
        oof_predictions[val_ind] = val_pred  # Add to out of folds array
        if test is not None:
            test_pred = model.predict(test[features]) # Predict the test set
            test_predictions += test_pred/n_folds
        # Compute fold metric
        score = amex_metric(y_val, val_pred)
        print(f'Our fold {fold} CV score is {score}')
        del x_train, x_val, y_train, y_val, lgb_train, lgb_valid
        gc.collect()
    score = amex_metric(train["target"], oof_predictions)  # Compute out of folds metric
    print(f'Our out of folds CV score is {score}')
    # Create a dataframe to store out of folds predictions
    oof_df = pd.DataFrame({'customer_ID': train['customer_ID'], 'target': train["target"], 'prediction': oof_predictions})
    oof_df.to_csv(OUTDIR+f'oof_lgbm_baseline_{n_folds}fold_seed{seed}.csv', index=False)
    # Create a dataframe to store test prediction
    if test is not None:
        test_df = pd.DataFrame({'customer_ID': test['customer_ID'], 'prediction': test_predictions})
        test_df.to_csv(f'/content/drive/MyDrive/Amex/Predictions/test_lgbm_baseline_{n_folds}fold_seed{seed}.csv', index = False)
    
    return model 

In [2]:
model = joblib.load(OUTDIR+f'lgbm13.pkl')

In [11]:
test_data = np.load(OUTDIR+"test_raw_all_mean_q1_q99_data.npy")


In [12]:
test_pred = model.predict(test_data.reshape(test_data.shape[0], -1)) # Predict the test set

In [13]:
import json 
with open(OUTDIR+'test_raw_all_mean_q1_q99_id.json', 'r') as f:
            test_id_dict = json.load(f)


In [14]:
result = pd.DataFrame({"customer_ID":test_id_dict.values(), 
                        "prediction":test_pred.reshape(-1)
                        }
                        )
    

In [15]:
import os 
sub_file_dir = os.path.join(OUTDIR, "lgbm_med_sub.csv")
result.set_index("customer_ID").to_csv(sub_file_dir)
